In [1]:
import smtplib
import email.message
import pandas as pd
from premailer import transform
import json

In [13]:
def send_mail(relatorio, destinatario, user, password):
    corpo_email = f"""
                    <title> Relatorio de Restaurantes </title>

                    <h1> Melhores Restaurantes </h1>
                    {relatorio}
                    """
    
    msg = email.message.Message()
    msg['subject'] = 'Relatorio de Restaurantes'
    msg['from'] = user
    msg['To'] = destinatario
    password = password
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(corpo_email)

    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()

    s.login(msg['From'], password)
    s.sendmail(msg['from'],msg['To'], msg.as_string().encode('utf-8'))
    print('Email enviado')

def rename_columns(dataframe):
        df = dataframe.copy()
        
        def snakecase(x):
            return x.replace(" ", "_").lower()

        cols_old = list(df.columns)

        cols_new = list(map(snakecase, cols_old))
        cols_old = list(map(str.strip, cols_old))  # Removendo espaços extras

        
        # Renomeando as colunas no dataframe
        df.columns = cols_new
        
        return df

def text_color(col):
    # Crie uma lista vazia para armazenar os estilos
    styles = [''] * len(col)
    
    # Aplique as regras de estilo conforme necessário
    for i, value in enumerate(col):
        styles[i] = 'color: black'  # Define a cor do texto como preto
    
    return styles


def background_color1(col):
    # Crie uma lista vazia para armazenar os estilos
    styles = ['background-color: #DCDCDC'] * len(col)  # Background em tom pastel
    
    return styles

def background_color2(col):
    # Crie uma lista vazia para armazenar os estilos
    styles = ['background-color: #F0E68C'] * len(col)  # Background em tom pastel
    
    return styles

def background_color3(col):
    # Crie uma lista vazia para armazenar os estilos
    styles = ['background-color: #B0C4DE'] * len(col)  # Background em tom pastel
    
    return styles

def background_color4(col):
    # Crie uma lista vazia para armazenar os estilos
    styles = ['background-color: #8FBC8F'] * len(col)  # Background em tom pastel
    
    return styles

def style_by_cost(custo, avg_cost):
    if avg_cost > 1500:
        return 'color: red'
    elif 700 <= avg_cost <= 1499:
        return 'color: yellow'
    else:
        return 'color: green'
    
def background_index1(col):
    styles = []
    for col_name in col:
        if col_name in ['restaurant_id', 'restaurant_name']:
            styles.append('background-color: #DCDCDC')
        elif col_name in ['country_code', 'city', 'cuisines']:
            styles.append('background-color: #F0E68C')   
        elif col_name in ['aggregate_rating', 'votes']:
            styles.append('background-color: #B0C4DE')  
        elif col_name in ['average_cost_for_two', 'custo']:
            styles.append('background-color: #8FBC8F')          
        else:
            styles.append('')
    return styles


In [3]:
with open('/home/thiago-vale/Documentos/GitHub/senha.json') as f:
    user = json.load(f)

In [4]:
df = pd.read_csv('../data/zomato.csv')
df = rename_columns(df)
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes'],
      dtype='object')

In [5]:
dados = df.loc[:,['restaurant_id','restaurant_name','country_code','city','cuisines','average_cost_for_two', 'aggregate_rating','votes']]\
                        .sort_values('aggregate_rating',ascending=False)

In [6]:
relatorio = dados.head(10)

In [7]:
relatorio['custo'] = '$'

/tmp/ipykernel_1340173/2144721183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relatorio['custo'] = '$'


In [8]:
relatorio = relatorio[['restaurant_id',
                        'restaurant_name',
                        'country_code', 
                        'city', 'cuisines',
                        'aggregate_rating', 
                        'votes',
                        'average_cost_for_two', 
                        'custo']]

In [14]:
# Aplicar o estilo ao DataFrame
styled_df = relatorio.style \
    .apply(background_color1, subset=pd.IndexSlice[:, ['restaurant_id', 'restaurant_name']]) \
    .apply(background_color2, subset=pd.IndexSlice[:, ['country_code', 'city', 'cuisines']]) \
    .apply(background_color3, subset=pd.IndexSlice[:, ['aggregate_rating', 'votes']]) \
    .apply(background_color4, subset=pd.IndexSlice[:, ['average_cost_for_two', 'custo']]) \
    .apply(lambda col: col.apply(lambda x: style_by_cost(x, col['average_cost_for_two'])), subset=['custo', 'average_cost_for_two'], axis=1) \
    .apply(text_color, subset=pd.IndexSlice[:, ['restaurant_id', 'restaurant_name', 'country_code', 'city', 'cuisines', 'average_cost_for_two', 'aggregate_rating', 'votes']]) \
    .apply_index(background_index1, axis='columns')\
    .apply_index(text_color, axis='columns')\
    .hide_index()

html_table = styled_df.render()
html_with_css = transform(html_table)


/tmp/ipykernel_1340173/2640549622.py:11: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  .hide_index()
/tmp/ipykernel_1340173/2640549622.py:13: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  html_table = styled_df.render()


In [10]:
send_mail(html_with_css, 'consultoria.thiagovale@gmail.com', user['email'], user['senha'])

Email enviado
